# **Kear Applications Tutorial**

In [2]:
from IPython.display import Image

In [4]:
#Image("/Users/jongil/ScreenShot/Screen Shot 2021-05-30 at 2.56.35 PM.png")

## **Usage examples for image classification models**

### Classify ImageNet classes with ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.kears.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print("Predicted: ", decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

### Extract features with VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprcessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

### Extract features from an arbitrary intermediate layer with VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

### Fine-tune InceptionV3 on a new set of classes

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.kears.preprocessing import image
from tensorflow.keras.models import Model
from tensroflow.keras.layers import Dense, GlobalAveragePooling2D

# if you have targe classes
#n = len(classes)

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePolling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activate='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activate='softmax')(x)
#predictions = Dense(n, activate='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convoultional IneptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the ramining top layers.

# let's visualize layer names and layer indices to see how many layers
# the first 239 layers and unfreeze the rest:
for layer in model.layer[:249]:
    layer.trainable = False
for layer in model.layer[249:]:
    layer.trainable = True
    
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train out model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(...)

### Build InceptionV3 over a custom input tensor

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input

# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(224, 224, 3))

model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)